In [9]:
import pandas as pd
import numpy as np

In [10]:
from collections import Counter

In [11]:
raw_data = pd.read_excel('8111B换流变A相油色谱监测数据.xlsx')
err_data = pd.read_excel('油色谱案例数据.xls')

In [12]:
def normalization(x):
    if x == '正常':
        x = 1
    else:
        x = 0
    return x

In [13]:
err_data['故障类型'] = err_data['故障类型'].apply(lambda x : normalization(x));

In [14]:
err_data['label'] = err_data.pop('故障类型')

In [15]:
Counter(err_data['label'])

Counter({0: 181, 1: 18})

In [17]:
err_data.insert(loc=5,column='total',value=err_data['CH4'] + err_data['C2H2'] + err_data['C2H4'] + err_data['C2H6'])

In [20]:
# err_data

In [21]:
CO = raw_data.pop('CO')
CO2 = raw_data.pop('CO2')
O2 = raw_data.pop('O2')
N2 = raw_data.pop('N2')
time = raw_data.pop('时间')
predicts = raw_data.pop('分类模型判别结果(0为正常点，1为异常点)')

In [22]:
raw_data['total'] = raw_data.pop('总烃')
raw_data['label'] = raw_data.pop(r'聚类模型判别结果(0为正常点，1为异常点)')

In [24]:
# raw_data

In [25]:
from decimal import Decimal
def isEqual(data):
    if abs(data.CH4 + data.C2H2 +  data.C2H4 +  data.C2H6 - data.total) < 1e-1:
        return True
    else:
        return False

In [26]:
r = raw_data.apply(lambda x:isEqual(x),axis=1)

In [27]:
Counter(r)

Counter({False: 252, True: 4209})

In [28]:
error_sample = raw_data[~r].reset_index(drop=True)

In [33]:
error_sample['error'] = error_sample.CH4 + error_sample.C2H2 + error_sample.C2H4 + error_sample.C2H6 - error_sample.total 

In [32]:
abs(error_sample.error).min()

0.10000000000000053

In [41]:
error_sample.error.apply(lambda x:abs(x)).max()

0.95100000000000051

In [ ]:
# 甲方说明：即使 总烃！=CH4 + C2H2 + C2H4 + C2H6 也不能说明设备异常
# 因此 首先按照阈值法和三比值法标注异常数据

In [38]:
# 原始监测数据中的C2H2最大值超过了阈值
raw_data.C2H2.max()

1.1519999999999999

In [34]:
warning_data = raw_data[raw_data.C2H2 - 0.8 > 1e-5]

,H2,CH4,C2H6,C2H4,C2H2,total,label
1383,15.000,8.600,1.600,1.100,1.000,12.500,0
1393,14.000,8.000,1.800,1.000,1.000,11.800,0
1403,14.000,8.800,1.100,1.100,1.000,12.500,0
2546,12.000,10.771,1.369,0.501,1.000,14.240,0
2561,11.000,9.718,1.448,0.408,1.000,12.924,0
4127,6.061,11.942,0.360,0.619,0.872,13.793,0
4230,6.061,11.722,1.822,0.333,1.152,15.028,1


In [36]:
# 原始监测数据中的H2最大值没有超过阈值
raw_data.H2.max()

15.199999999999999

In [42]:
warning_data = raw_data[raw_data.C2H2 - 0.8 > 1e-5]